### Read in TIMS Data from S3 Bucket

**June 11, 2019**

### S3 buckets

+ Each project has a S3 bucket with the data of your partner. 
+ **Only** the team, PM, TM and your partner have access to that.

![](images/s3_buckets.png)

**Requirements to access your bucket**

+ Have a pair of credentials from AWS -> I will send you your credentials.
+ Install [AWSCLI](https://aws.amazon.com/cli/) `pip install awscli` (do it on your dssg virutal environment)
+ The name of your S3 bucket:
    + air-pollution-uk
    
#### Setup your AWS credentials 

Check if you have a file `credentials` on `~/.aws/`, if you dont create that file `nano credentials` (no extension), if you already have one at the end of it add a name to your dssg profile. 

```
[dssg]
aws_access_key_id = your_access_key_id
aws_secret_access_key = your_secret_access_key
```
    
You can access your S3 bucket through **S3FS** or **Boto**. 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys, os
import boto3
import OSGridConverter

### Get TIMS data from S3 Bucket


In [3]:
session = boto3.Session(profile_name='dssg')
# you can also put your credentials directly... through a json file
#session = boto3.Session(aws_access_key_id=credentials['aws_access_key_id'], 
#                       aws_secret_access_key=credentials['aws_secret_access_key'])

s3 = session.client('s3')
print(type(s3))

bucket_name = 'air-pollution-uk'

<class 'botocore.client.S3'>


In [4]:
obj = s3.get_object(Bucket=bucket_name, Key='raw/tims_data/detdata01032019-204523.csv')
initial_df = pd.read_csv(obj['Body']) # 'Body' is a key word


### Initial Data Exploration 

#### Data Description

1. TIMESTAMP

2. NODE

3. EASTING: each datapoint for every node has same easting/northing info

4. NORTHING:

5. FLOW_ACTUAL_15M = Average flows for past 15minutes over each working detector which belongs to a given node & for which the data is available (a moving average) 

6. SAT_BANDINGS = Average Saturation for past 15minutes from all working detectors which belong to a given node &amp; for which the data is available  converted to bands:

 - 0-79%
 -  80-89%
 -  90-99%
 - $>=$100%
 - No Data

7. DETECTOR_No – Number of detectors that belong to each node & that data is available

8. TOTAL_DETECTOR_NO – Total No of detectors that belong to each node

9. DETECTOR_Rate = % of detectors from each node the data comes from = (DETECTOR_No/ TOTAL_DETECTOR_NO )

#### Questions: 
1. How is the moving average computed? Perhaps by averaging over number of working detectors? 
2. What is the reference point for the easting/northings? Is this based off of the National Grid Reference? Is the name of the mapping datum (to convert to lat/long) WGS84? 
3. What is saturation?

In [44]:
initial_df.head()

,TIMESTAMP,NODE,EASTING,NORTHING,FLOW_ACTUAL_15M,SAT_BANDINGS,DETECTOR_NO,TOTAL_DETECTOR_NO,DETECTOR_RATE
0,2019-03-01T20:31:00Z,29/013,513643.42,185553.97,459,0-79%,5,5,1.0
1,2019-03-01T20:31:00Z,29/020,513010.00,187530.00,572,0-79%,3,3,1.0
2,2019-03-01T20:31:00Z,29/028,514272.61,186392.99,456,0-79%,2,2,1.0
3,2019-03-01T20:31:00Z,29/035,517233.36,192368.23,291,0-79%,6,6,1.0
4,2019-03-01T20:31:00Z,29/046,517103.35,192319.26,572,0-79%,2,2,1.0


In [14]:
print(initial_df.shape)
print(initial_df.columns.tolist())
print(initial_df.dtypes)

(49043, 9)
['TIMESTAMP', 'NODE', 'EASTING', 'NORTHING', 'FLOW_ACTUAL_15M', 'SAT_BANDINGS', 'DETECTOR_NO', 'TOTAL_DETECTOR_NO', 'DETECTOR_RATE']
TIMESTAMP             object
NODE                  object
EASTING              float64
NORTHING             float64
FLOW_ACTUAL_15M        int64
SAT_BANDINGS          object
DETECTOR_NO            int64
TOTAL_DETECTOR_NO      int64
DETECTOR_RATE        float64
dtype: object


In [51]:
initial_df.describe()

,EASTING,NORTHING,FLOW_ACTUAL_15M,DETECTOR_NO,TOTAL_DETECTOR_NO,DETECTOR_RATE
count,49043.000000,49043.000000,49043.000000,49043.000000,49043.000000,49043.000000
mean,530245.937163,180233.572091,403.880717,2.948107,3.273739,0.914365
std,12439.594438,7188.446131,203.206044,1.882281,2.001284,0.182434
min,0.000000,0.000000,0.000000,1.000000,1.000000,0.170000
25%,525474.110000,176684.980000,272.000000,2.000000,2.000000,1.000000
50%,530445.600000,180698.080000,378.000000,2.000000,3.000000,1.000000
75%,535523.000000,184346.390000,502.000000,4.000000,4.000000,1.000000
max,556075.790000,199720.000000,3110.000000,18.000000,18.000000,1.000000


In [13]:
initial_df[["NODE","FLOW_ACTUAL_15M", "DETECTOR_NO", "TOTAL_DETECTOR_NO", "DETECTOR_RATE"]].groupby(["NODE"]).describe()

FLOW_ACTUAL_15M                                                   \
                 count         mean        std     min      25%     50%   
NODE                                                                      
00/002            14.0   357.285714  17.490343   326.0   344.75   356.5   
00/003            14.0   123.285714   5.594346   112.0   120.00   125.0   
00/004            14.0   330.071429   5.413150   321.0   325.50   330.0   
00/005            14.0   662.285714   9.972490   650.0   653.25   661.0   
00/006            14.0   288.214286  14.175703   269.0   277.00   286.5   
00/007            14.0   133.428571   8.391518   121.0   128.25   130.5   
00/008            14.0   304.285714   6.354024   298.0   299.00   302.0   
00/009            14.0   255.571429   3.975198   248.0   254.00   255.0   
00/010            14.0   181.071429   5.151016   174.0   176.00   181.5   
00/011            14.0   199.214286   8.648509   187.0   194.25   196.0   
00/012            14.0   159.428571  16.951126   142.0   145.50   150.0   
00/013            14.0   159.642857  16.462578   141.0   147.75   153.0   
00/014            14.0   324.142857  15.859271   306.0   310.50   320.0   
00/015            14.0   493.785714  20.374111   462.0   478.25   491.5   
00/016            14.0   268.142857   4.537996   262.0   264.25   267.5   
00/019            14.0   283.357143  12.767791   260.0   279.75   285.0   
00/020            14.0   248.071429  13.487683   229.0   239.25   245.5   
00/021            14.0   294.071429  17.220133   272.0   278.75   291.5   
00/022            14.0   133.928571  19.384626    98.0   117.75   140.5   
00/023            14.0   209.000000  10.575734   195.0   199.50   206.0   
00/024            14.0    43.857143   3.958577    32.0    44.00    45.0   
00/025            14.0   346.142857  19.166706   323.0   329.25   338.5   
00/027            14.0   454.357143  33.894876   399.0   432.75   452.0   
00/028            14.0   204.357143   6.617077   190.0   202.25   205.5   
00/029            14.0   225.571429   8.924593   212.0   220.50   224.5   
00/030            14.0   406.714286  14.896714   384.0   390.50   410.0   
00/031            14.0   269.500000   4.988448   258.0   267.25   270.0   
00/032            14.0   136.571429   9.936059   127.0   128.25   134.0   
00/033            14.0   197.571429  17.270314   182.0   184.50   189.5   
00/035            14.0   506.642857  22.544340   476.0   486.50   509.5   
...                ...          ...        ...     ...      ...     ...   
32/124            13.0   437.692308  12.378642   414.0   428.00   438.0   
32/125            13.0   186.538462   9.143444   169.0   181.00   188.0   
32/129            13.0   327.692308  10.339122   307.0   323.00   329.0   
32/132            13.0   417.538462   5.316882   407.0   415.00   418.0   
32/133            13.0   466.692308  10.530785   452.0   462.00   465.0   
32/143            13.0   189.769231   6.482719   177.0   187.00   190.0   
32/154            14.0   262.428571   7.772875   247.0   258.00   260.5   
32/157            12.0   532.250000  53.926086   456.0   499.00   530.0   
32/169            14.0   422.142857  15.737545   398.0   411.25   419.0   
32/170            14.0   403.500000  10.420025   386.0   398.50   404.5   
32/175            13.0   346.230769  14.254554   326.0   334.00   355.0   
32/176            13.0   274.615385   8.722944   262.0   268.00   272.0   
32/178            13.0   486.000000  25.544080   439.0   470.00   494.0   
32/179            13.0   583.076923  15.622961   538.0   584.00   587.0   
32/180            13.0   519.000000   7.937254   504.0   514.00   522.0   
32/182            13.0   412.923077  23.865811   388.0   393.00   400.0   
32/184            13.0   528.384615  22.831040   483.0   520.00   530.0   
32/185            13.0   886.384615  15.218073   850.0   877.00   888.0   
32/186            13.0   463.846154  15.377723   432.0   454.00   466.0   
32/187            13.0   335.615

In [59]:
np.unique(initial_df.groupby(["NODE"]).size())

array([ 1,  3,  6,  7,  9, 10, 11, 12, 13, 14, 15], dtype=int64)

In [9]:
print(np.unique(initial_df["NODE"]))
print((np.unique(initial_df["NODE"]).shape))

['00/002' '00/003' '00/004' ... '32/228' '32/229' '32/999']
(3587,)


### Data Cleaning

In [16]:
#convert easting/northing to lat long based on nat grid ref
#https://pypi.org/project/OSGridConverter/

initial_df.EASTING = initial_df.EASTING.astype(str)
initial_df.NORTHING = initial_df.NORTHING.astype(str)

print(initial_df.dtypes)

TIMESTAMP             object
NODE                  object
EASTING               object
NORTHING              object
FLOW_ACTUAL_15M        int64
SAT_BANDINGS          object
DETECTOR_NO            int64
TOTAL_DETECTOR_NO      int64
DETECTOR_RATE        float64
dtype: object


In [19]:
initial_df["EASTINGNORTHING"] = "TG " + initial_df["EASTING"] + " " + initial_df["NORTHING"]
initial_df.head()

,TIMESTAMP,NODE,EASTING,NORTHING,FLOW_ACTUAL_15M,SAT_BANDINGS,DETECTOR_NO,TOTAL_DETECTOR_NO,DETECTOR_RATE,EASTINGNORTHING
0,2019-03-01T20:31:00Z,29/013,513643.42,185553.97,459,0-79%,5,5,1.0,TG 513643.42 185553.97
1,2019-03-01T20:31:00Z,29/020,513010.0,187530.0,572,0-79%,3,3,1.0,TG 513010.0 187530.0
2,2019-03-01T20:31:00Z,29/028,514272.61,186392.99,456,0-79%,2,2,1.0,TG 514272.61 186392.99
3,2019-03-01T20:31:00Z,29/035,517233.36,192368.23,291,0-79%,6,6,1.0,TG 517233.36 192368.23
4,2019-03-01T20:31:00Z,29/046,517103.35,192319.26,572,0-79%,2,2,1.0,TG 517103.35 192319.26


In [21]:
def grid_converter(grid):
    '''wrapper function for converting national grid coordinates to lat long'''
# grid = 'TG 51408 13177'
    l = grid2latlong(grid = grid)
    return np.float64(l.latitude), np.float64(l.longitude)

# initial_df[['latitude', 'longitude']] = initial_df.apply(lambda x: grid2latlong(x.EASTINGNORTHING))

initial_df['latitude'], initial_df['longitude'] = zip(*initial_df['EASTINGNORTHING'].map(grid2latlong))
initial_df.head()

NameError: name 'grid2latlong' is not defined

5.0


Junk code, to be deleted later 

In [11]:
#junk code

lines = obj['Body'].read().decode('utf-8').split()

# now iterate over those lines
for row in csv.DictReader(lines):
    print("lines")
    # here you get a sequence of dicts
    # do whatever you want with each line here
    print(row)

### References 

+ [S3FS Documentation](https://s3fs.readthedocs.io/en/latest/)
+ [Setup credentials AWS documentation](https://docs.aws.amazon.com/sdk-for-java/v1/developer-guide/setup-credentials.html)